In [1]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read().split('\n')

# Part 1

In [2]:
def trajectory_to_actions(trajectory):
    ''' split the string of movements into a list of tiles to move as int and a rotation as string
        e.g. 10R5L2 -> [10, 'R', 5, 'L', 2] '''
    actions = []
    last = []
    for c in trajectory:
        try:
            int(c)
            last.append(c)
        except ValueError:
            actions.append(int(''.join(last)))
            last = []
            actions.append(c)
    actions.append(int(''.join(last)))
    return actions

def get_nodes(field_list, n):
    ''' from a list of lists that is the read data, this creates a dict of the nodes that are present. '''
    nodes = dict()
    for imag in range(M):
        for real in range(N):
            c = field_list[imag][real]
            if c == '.':
                nodes[complex(real, imag)] = ['open', (imag // n, real // n)]
            elif c == '#':
                nodes[complex(real, imag)] = ['solid', (imag // n, real // n)]
    return nodes

In [3]:
rotations = {'R': {'n': 'e', 'e': 's', 's': 'w', 'w': 'n'},
             'L': {'n': 'w', 'w': 's', 's': 'e', 'e': 'n'}}
facings = {'n': complex(0, -1), 's': complex(0, 1), 'w': complex(-1, 0), 'e': complex(1, 0)}

class myself:
    def __init__(self, start_location, start_facing, nodes):
        self.state = dict()
        self.location = start_location
        self.facing = start_facing
        
        self.nodes = nodes
    
    def perform_action(self, action):
        if isinstance(action, int):
            self.move(action)
        else:
            self.rotate(action)
    
    def move(self, steps):
        for _ in range(steps):
            to_move = self.location + facings[self.facing]
            new_facing = self.facing
            
            if not to_move in self.nodes:
                to_move, new_facing = wrap_around(self.location, self.facing, self.nodes)
                
            if nodes[to_move][0] == 'solid':
                pass
            
            else:
                self.location = to_move
                self.facing = new_facing
            
    def rotate(self, direction):
        self.facing = rotations[direction][self.facing]
        
    def __repr__(self):
        return f'loc: {self.location}, facing: {self.facing}'

In [4]:
options = {'n': [max, 'real'], 
           's': [min, 'real'],
           'w': [max, 'imag'],
           'e': [min, 'imag']}
other_attr = {'real': 'imag',
              'imag': 'real'}

def wrap_around(node, facing, nodes):
    func, attr = options[facing]
    same_row_or_col = [c for c in nodes.keys() if getattr(c, attr) == getattr(node, attr)] 
    return func(same_row_or_col, key = lambda c: getattr(c, other_attr[attr])), facing

In [5]:
x = get_input('test')
trajectory = x[-1]
field_list = [list(row)+[' ']*(len(max(x, key=len))- len(row)) for row in x[:-2]]
M, N = len(field_list), len(field_list[0])

In [6]:
actions = trajectory_to_actions(trajectory)
nodes = get_nodes(field_list, 4)

top_row = min(nodes.keys(), key=lambda c: c.imag).imag
start_node = min([c for c in nodes if c.imag == top_row], key=lambda c: c.real)
start_facing = 'e'

In [7]:
me = myself(start_node, start_facing, nodes)

for action in actions:
    me.perform_action(action)

facing_map = {'e': 0, 's': 1, 'w': 2, 'n': 3}
print( 1000*(me.location.imag+1) + 4*(me.location.real + 1) + facing_map[me.facing])

6032.0


# Part 2



In [13]:
test_cube = {
    (0, 2): {
        'n': [(1,0), 'n', 'odd'],
        'e': [(2,3), 'e', 'odd'],
        's': [(1,2), 'n', 'even'],
        'w': [(1,1), 'n', 'even'],
    },
    (1, 0): {
        'n': [(0,2), 'n', 'odd'],
        'e': [(1,1), 'w', 'even'],
        's': [(2,2), 's', 'odd'],
        'w': [(2,3), 's', 'odd'],
    },
    (1, 1): {
        'n': [(0,2), 'w', 'even'],
        'e': [(1,2), 'w', 'even'],
        's': [(2,2), 'w', 'odd'],
        'w': [(1,0), 'e', 'even'],
    },
    (1, 2): {
        'n': [(0,2), 's', 'even'],
        'e': [(2,3), 'n', 'odd'],
        's': [(2,2), 'n', 'even'],
        'w': [(1,1), 'e', 'even'],
    },
    (2, 2): {
        'n': [(1,2), 's', 'even'],
        'e': [(2,3), 'w', 'even'],
        's': [(1,0), 's', 'odd'],
        'w': [(1,1), 's', 'odd'],
    },
    (2, 3): {
        'n': [(1,2), 'e', 'odd'],
        'e': [(0,2), 'e', 'odd'],
        's': [(1,0), 'w', 'odd'],
        'w': [(2,2), 'e', 'even'],
    },
}

input_cube = {
    (0, 1): {
        'n': [(3,0), 'w', 'even'],
        'w': [(2,0), 'w', 'odd'],
    },
    (0, 2): {
        'n': [(3,0), 's', 'even'],
        'e': [(2,1), 'e', 'odd'],
        's': [(1,1), 'e', 'even'],
    },
    (1, 1): {
        'e': [(0,2), 's', 'even'],
        'w': [(2,0), 'n', 'even'],
    },
    (2, 0): {
        'n': [(1,1), 'w', 'even'],
        'w': [(0,1), 'w', 'odd'],
    },
    (2, 1): {
        'e': [(0,2), 'e', 'odd'],
        's': [(3,0), 'e', 'even'],
    },
    (3, 0): {
        'w': [(0,1), 'n', 'even'],
        'e': [(2,1), 's', 'even'],
        's': [(0,2), 'n', 'even'],
    },
}

options = {'n': [min, 'imag'],
           'e': [max, 'real'],
           's': [max, 'imag'],
           'w': [min, 'real']}

other_attr = {'real': 'imag',
              'imag': 'real'}

new_facings = {'n': 's',
               'e': 'w',
               's': 'n',
               'w': 'e'}

def get_tiles(nodes):
    tiles = dict()
    for node,info in nodes.items():
        if info[1] not in tiles:
            tiles[info[1]] = [node]
        else:
            tiles[info[1]].append(node)
    return tiles


def get_sorted_edge(tile, edge):
    func, attr = options[edge]
    tile_nodes = tiles[tile]
    desired_attr = getattr(func(tile_nodes, key=lambda c: getattr(c, attr)), attr)
    return sorted([t for t in tile_nodes if getattr(t, attr) == desired_attr], key=lambda c: getattr(c, other_attr[attr]))
    
        
def wrap_around(node, facing, nodes):
    this_tile = nodes[node][1]
    index = get_sorted_edge(this_tile, facing).index(node)
    
    
    other_tile, other_edge, parity = input_cube[this_tile][facing]
    candidate_tiles = get_sorted_edge(other_tile, other_edge)
    
    new_facing = new_facings[other_edge]
    if parity == 'even':
        return candidate_tiles[index], new_facing
    else:
        return candidate_tiles[::-1][index], new_facing
    

In [15]:
x = get_input('input')
trajectory = x[-1]
field_list = [list(row)+[' ']*(len(max(x, key=len))- len(row)) for row in x[:-2]]
M, N = len(field_list), len(field_list[0])

In [16]:
actions = trajectory_to_actions(trajectory)
nodes = get_nodes(field_list, 50)
tiles = get_tiles(nodes)

top_row = min(nodes.keys(), key=lambda c: c.imag).imag
start_node = min([c for c in nodes if c.imag == top_row], key=lambda c: c.real)
start_facing = 'e'

In [17]:
me = myself(start_node, start_facing, nodes)

for action in actions:
    me.perform_action(action)

facing_map = {'e': 0, 's': 1, 'w': 2, 'n': 3}
print( 1000*(me.location.imag+1) + 4*(me.location.real + 1) + facing_map[me.facing])

19534.0
